## Download Library

In [1]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
import openai

In [3]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.8 MB/s eta 0:00:00


## Load `openassitant-guanaco` Data as Demonstration

We want to understand the data format.

### Code

- **Importing `load_dataset`**: The `from datasets import load_dataset` statement imports the `load_dataset` function from the `datasets` library. This library is a part of the Hugging Face ecosystem, designed to easily share, load, and work with datasets in the machine learning field.

- **Loading the Dataset**: The `load_dataset("timdettmers/openassistant-guanaco")` function call tells the library to load a dataset identified by the name `timdettmers/openassistant-guanaco`. This identifier typically consists of the username or organization name (`timdettmers` in this case) and the specific dataset name (`openassistant-guanaco`).

In [38]:
from datasets import load_dataset

dataset = load_dataset("timdettmers/openassistant-guanaco")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [39]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9846
    })
    test: Dataset({
        features: ['text'],
        num_rows: 518
    })
})

In [40]:
dataset["train"]

Dataset({
    features: ['text'],
    num_rows: 9846
})

In [41]:
dataset["train"][0]

{'text': '### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining po

In [42]:
type(dataset["train"][0])

dict

In [43]:
dataset["train"][0].keys()

dict_keys(['text'])

In [44]:
dataset["train"][1]["text"]

'### Human: ¿CUales son las etapas del desarrollo y en qué consisten según Piaget?### Assistant: Jean Piaget fue un psicólogo suizo que propuso una teoría sobre el desarrollo cognitivo humano que consta de cuatro etapas:\n\nEtapa sensoriomotora (0-2 años): Durante esta etapa, el niño aprende a través de sus sentidos y movimientos. Descubre que sus acciones pueden tener un impacto en el entorno y comienza a formarse una idea básica de objetividad y continuidad.\n\nEtapa preoperatoria (2-7 años): En esta etapa, el niño comienza a desarrollar un pensamiento simbólico y a comprender que las cosas pueden representar a otras cosas. También comienzan a desarrollar un pensamiento lógico y a comprender conceptos como la causa y el efecto.\n\nEtapa de operaciones concretas (7-12 años): Durante esta etapa, el niño desarrolla un pensamiento lógico y comprende las relaciones causales. Empiezan a comprender que las cosas pueden tener múltiples perspectivas y que los conceptos pueden ser más complejo

In [45]:
type(dataset["train"][0]["text"])

str

In [46]:
len(dataset["train"][0]["text"])

1687

## Scrap `Any` PDF

We need the `PyMuPDF` package in python. So, we install it first.

In [4]:
! pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 55.6 MB/s eta 0:00:00


### `read_pdf_content` function

In [5]:
import fitz  # PyMuPDF

def read_pdf_content(pdf_path):
    """
    Reads a PDF and returns its content as a list of strings.

    Args:
    pdf_path (str): The file path to the PDF.

    Returns:
    list of str: A list where each element is the text content of a PDF page.
    """
    content_list = []
    with fitz.open(pdf_path) as doc:
        for page in doc:
            content_list.append(page.get_text())

    return content_list

In [6]:
%%time

scraped_content = read_pdf_content("all_ysa_doc.pdf")

CPU times: user 49.4 ms, sys: 3.32 ms, total: 52.8 ms
Wall time: 52.5 ms


In [7]:
len(scraped_content[0])

237

In [8]:
scraped_content = ' '.join(scraped_content)

In [9]:
len(scraped_content)

28995

In [10]:
type(scraped_content)

str

In [11]:
scraped_content.split('. ')[0]

'About YSA\nYouth Spirit Artworks (YSA) is a program in Berkeley, California, that helps homeless and\nlow-income young people in the San Francisco Bay Area'

## API Call to Create Data

Here we use the `client.chat.completions.create` function from *OpenAI* as a helper function to assist us to create question answer.

In [12]:
OPENAI_API_KEY = "sk-xxx"
openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)


def call_chatgpt(query: str, model: str = "gpt-3.5-turbo") -> str:
    """
    Generates a response to a query using the specified language model.
    Args:
        query (str): The user's query that needs to be processed.
        model (str, optional): The language model to be used. Defaults to "gpt-3.5-turbo".
    Returns:
        str: The generated response to the query.
    """

    # Prepare the conversation context with system and user messages.
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Question: {query}."},
    ]

    # Use the OpenAI client to generate a response based on the model and the conversation context.
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )

    # Extract the content of the response from the first choice.
    content: str = response.choices[0].message.content

    # Return the generated content.
    return content

In [13]:
resp = call_chatgpt("tell me a joke")

In [14]:
resp

"Sure, here's a classic one for you: \n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"

### Prompt Engineer

We use prompt engineer to ensure the content `GPT` gave us is in the same content as the `openassist/guanaco` data.

```python
    ### Human:
    ### Assistant:
```

In [15]:
def prompt_engineered_api(text: str):

    prompt = f"""
        I have the following content: {text}

        I want to create a question-answer content that has the following format:

        ### Human:
        ### Assistant:

        Make sure to write question and answer based on the content I provided.

        The ### Human means it's a question, and the ### Assistant means it's an answer.
    """

    resp = call_chatgpt(prompt)

    return resp

In [47]:
scraped_content.split('. ')[0]

'About YSA\nYouth Spirit Artworks (YSA) is a program in Berkeley, California, that helps homeless and\nlow-income young people in the San Francisco Bay Area'

In [16]:
resp = prompt_engineered_api(scraped_content.split('. ')[0])

resp

'### Human: What is Youth Spirit Artworks (YSA)?\n\n### Assistant: Youth Spirit Artworks (YSA) is a program in Berkeley, California, that helps homeless and low-income young people in the San Francisco Bay Area.'

In [17]:
type(resp)

str

## Create `DatasetDict` Data Structure

```python
from datasets import Dataset, DatasetDict

# Example data - replace these with your actual data
train_data = {'text': [resp]*3}
test_data = {'text': [resp]*2}

# Create Dataset objects for training and testing
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

# Combine them into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

# Display the structure of the dataset
print(dataset_dict)
```

In [19]:
from datasets import Dataset, DatasetDict
from tqdm import tqdm

In [20]:
raw_content_for_train = []

for i in tqdm(range(len(scraped_content.split('. ')))):
    resp = prompt_engineered_api(scraped_content.split('. ')[i])
    raw_content_for_train.append(resp)

100%|██████████| 131/131 [02:45<00:00,  1.26s/it]


In [26]:
raw_content_for_train[0]

'### Human: What does YSA stand for?\n### Assistant: YSA stands for Youth Spirit Artworks.'

In [25]:
# Example data - replace these with your actual data
train_data = {'text': raw_content_for_train[0:100]}
test_data = {'text': raw_content_for_train[100::]}

# Create Dataset objects for training and testing
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

# Combine them into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

# Display the structure of the dataset
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text'],
        num_rows: 31
    })
})


In [27]:
dataset_dict["train"][0]["text"]

'### Human: What does YSA stand for?\n### Assistant: YSA stands for Youth Spirit Artworks.'

## Push to HuggingFace Hub

In [28]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [30]:
from huggingface_hub import HfApi, create_repo

In [33]:
# Replace 'your_token_here' with your actual Hugging Face Auth token
# Replace 'youthless-homeless-shelter-web-scrape-dataset' with your desired repository name
auth_token = 'xxx'
repo_name = 'youthless-homeless-shelter-web-scrape-dataset'
username = 'eagle0504' # replace with your Hugging Face username

api = HfApi()
create_repo(repo_name, token=auth_token, private=False) # Set private=True if you want it to be a private dataset


RepoUrl('https://huggingface.co/eagle0504/youthless-homeless-shelter-web-scrape-dataset', endpoint='https://huggingface.co', repo_type='model', repo_id='eagle0504/youthless-homeless-shelter-web-scrape-dataset')

In [36]:
app_id = f"{username}/{repo_name}"
print(app_id)

eagle0504/youthless-homeless-shelter-web-scrape-dataset


In [37]:
dataset_dict.push_to_hub(app_id)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/eagle0504/youthless-homeless-shelter-web-scrape-dataset/commit/715e0de1503e27d35056393f2bdb8fa3707c1cb3', commit_message='Upload dataset', commit_description='', oid='715e0de1503e27d35056393f2bdb8fa3707c1cb3', pr_url=None, pr_revision=None, pr_num=None)